**Nuclei segmentation in whole slide H&E images**

using custom model.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from tifffile import imread, imwrite
from csbdeep.utils import Path, normalize
from csbdeep.utils.tf import keras_import
keras = keras_import()

from stardist import export_imagej_rois, random_label_cmap
from stardist.models import StarDist2D
import tensorflow as tf

np.random.seed(0)
cmap = random_label_cmap()

In [2]:
if tf.config.list_physical_devices('GPU'):
    print("GPU is available")
else:
    print("GPU is not available")

print(tf.__version__)

GPU is available
2.10.1


In [3]:
import os
from glob import glob

# CHANGE PATH

pths = [r'\\10.162.80.16\Andre\data\Ashleigh fallopian tube\fallopian tubes\AJER376',
        r'\\10.162.80.16\Andre\data\Ashleigh fallopian tube\fallopian tubes\AJGB283',
        r'\\10.162.80.16\Andre\data\Ashleigh fallopian tube\fallopian tubes\AJF232',
        r'\\10.162.80.16\Andre\data\Ashleigh fallopian tube\fallopian tubes\AJLC170']

# pth = r'\\10.99.68.178\andreex\data\monkey fetus\gestational 40'  # path of ndpi files
WSIs = [glob(f'{pth}\*.ndpi') for pth in pths]

In [4]:
len(WSIs)

4

In [5]:
len(WSIs[1])

300

In [6]:
WSIs[:20]

[['\\\\10.162.80.16\\Andre\\data\\Ashleigh fallopian tube\\fallopian tubes\\AJER376\\fallopian_tubes_AJER376_001.ndpi',
  '\\\\10.162.80.16\\Andre\\data\\Ashleigh fallopian tube\\fallopian tubes\\AJER376\\fallopian_tubes_AJER376_003.ndpi',
  '\\\\10.162.80.16\\Andre\\data\\Ashleigh fallopian tube\\fallopian tubes\\AJER376\\fallopian_tubes_AJER376_009.ndpi',
  '\\\\10.162.80.16\\Andre\\data\\Ashleigh fallopian tube\\fallopian tubes\\AJER376\\fallopian_tubes_AJER376_013.ndpi',
  '\\\\10.162.80.16\\Andre\\data\\Ashleigh fallopian tube\\fallopian tubes\\AJER376\\fallopian_tubes_AJER376_015.ndpi',
  '\\\\10.162.80.16\\Andre\\data\\Ashleigh fallopian tube\\fallopian tubes\\AJER376\\fallopian_tubes_AJER376_017.ndpi',
  '\\\\10.162.80.16\\Andre\\data\\Ashleigh fallopian tube\\fallopian tubes\\AJER376\\fallopian_tubes_AJER376_019.ndpi',
  '\\\\10.162.80.16\\Andre\\data\\Ashleigh fallopian tube\\fallopian tubes\\AJER376\\fallopian_tubes_AJER376_021.ndpi',
  '\\\\10.162.80.16\\Andre\\data\\Ashlei

Load just 1 image and show (takes a couple min)

In [7]:
def show_image(img, crop_x, crop_y, tile_size, **kwargs):
    """Plot large image at different resolutions."""
    fig, ax = plt.subplots(1, 2, figsize=(16, 8))

    # Plot the original image on the left
    ax[0].imshow(img, **kwargs)

    # Crop the image
    cropped_img = img[crop_y:crop_y+tile_size, crop_x:crop_x+tile_size]

    # Plot the cropped image on the right
    ax[1].imshow(cropped_img, **kwargs)

    ax[0].axis('off')
    ax[1].axis('off')

    plt.tight_layout()
    plt.show()

In [8]:
import json
from stardist.models import Config2D, StarDist2D

def load_model(model_path: str) -> StarDist2D:
    # Load StarDist model weights, configurations, and thresholds
    with open(model_path + '\\config.json', 'r') as f:
        config = json.load(f)
    with open(model_path + '\\thresholds.json', 'r') as f:
        thresh = json.load(f)
    model = StarDist2D(config=Config2D(**config), basedir=model_path, name='offshoot_model')
    model.thresholds = thresh
    print('Overriding defaults:', model.thresholds, '\n')
    model.load_weights(model_path + '\\weights_best.h5')
    return model

#
# CHANGE RIGHT MODEL
# model = load_model(r"\\10.99.68.178\andreex\data\Stardist\qupath_training_annotations\models\monkey_ft_11_21_2023_lr_0.001_epochs_400_pt_40")
model = load_model(r"\\10.99.68.178\andreex\data\Stardist\qupath_training_annotations\models\fallopian_tube_12_7_2023_lr_0.001_epochs_400_pt_40")


base_model.py (149): output path for model already exists, files may be overwritten: \\10.99.68.178\andreex\data\Stardist\qupath_training_annotations\models\fallopian_tube_12_7_2023_lr_0.001_epochs_400_pt_40\offshoot_model


Using default values: prob_thresh=0.5, nms_thresh=0.4.
Overriding defaults: Thresholds(prob=0.5534873561706642, nms=0.3) 


To save prediction outputs, I save as geojson file.
By reading this template geojson file, I get the right type info that I need to save

In [9]:
import json

def save_json(result, out_pth, name):
    """Saves a geojson file with centroids and contours for StarDist output. Useful for loading into qupath"""
    coords = result['coord']
    points = result['points']

    json_data = []

    for i in range(len(points)):
        point = points[i]
        contour = coords[i]
        centroid = [int(point[0]), int(point[1])]
        contour = [[float(coord) for coord in xy[::-1]] for xy in contour]

        # Create a new dictionary for each contour
        dict_data = {
            "centroid": [centroid],
            "contour": [contour]
        }

        json_data.append(dict_data)

    new_fn = name[:-5] + '.json'

    with open(os.path.join(out_pth, new_fn),'w') as outfile:
        json.dump(json_data, outfile)
    print('Finished',new_fn)

In [10]:
# Name of output folder
date = '12_11_23'

for pth in pths:
    out_pth = os.path.join(pth,f'StarDist_{date}')
    if not os.path.exists(out_pth):
        os.mkdir(out_pth)
    
     
    out_pth_json = os.path.join(out_pth,'json')
    out_pth_tif = os.path.join(out_pth,'tif')
    print(out_pth_json)
    
    if not os.path.exists(out_pth_json):
        os.mkdir(out_pth_json)
    
    if not os.path.exists(out_pth_tif):
        os.mkdir(out_pth_tif)

\\10.162.80.16\Andre\data\Ashleigh fallopian tube\fallopian tubes\AJER376\StarDist_12_11_23\json
\\10.162.80.16\Andre\data\Ashleigh fallopian tube\fallopian tubes\AJGB283\StarDist_12_11_23\json
\\10.162.80.16\Andre\data\Ashleigh fallopian tube\fallopian tubes\AJF232\StarDist_12_11_23\json
\\10.162.80.16\Andre\data\Ashleigh fallopian tube\fallopian tubes\AJLC170\StarDist_12_11_23\json


In [11]:
model

StarDist2D(offshoot_model): YXC → YXC
├─ Directory: \\10.99.68.178\andreex\data\Stardist\qupath_training_annotations\models\fallopian_tube_12_7_2023_lr_0.001_epochs_400_pt_40\offshoot_model
└─ Config2D(n_dim=2, axes='YXC', n_channel_in=3, n_channel_out=33, train_checkpoint='weights_best.h5', train_checkpoint_last='weights_last.h5', train_checkpoint_epoch='weights_now.h5', n_rays=32, grid=(2, 2), backbone='unet', n_classes=None, unet_n_depth=3, unet_kernel_size=[3, 3], unet_n_filter_base=32, unet_n_conv_per_depth=2, unet_pool=[2, 2], unet_activation='relu', unet_last_activation='relu', unet_batch_norm=False, unet_dropout=0.0, unet_prefix='', net_conv_after_unet=128, net_input_shape=[None, None, 3], net_mask_shape=[None, None, 1], train_shape_completion=False, train_completion_crop=32, train_patch_size=[256, 256], train_background_reg=0.0001, train_foreground_only=0.9, train_sample_cache=True, train_dist_loss='mae', train_loss_weights=[1, 0.2], train_class_weights=[1, 1], train_epochs=40

In [12]:
for folder in WSIs:
    print(folder)
    break
    #or img_pth in folder:

['\\\\10.162.80.16\\Andre\\data\\Ashleigh fallopian tube\\fallopian tubes\\AJER376\\fallopian_tubes_AJER376_001.ndpi', '\\\\10.162.80.16\\Andre\\data\\Ashleigh fallopian tube\\fallopian tubes\\AJER376\\fallopian_tubes_AJER376_003.ndpi', '\\\\10.162.80.16\\Andre\\data\\Ashleigh fallopian tube\\fallopian tubes\\AJER376\\fallopian_tubes_AJER376_009.ndpi', '\\\\10.162.80.16\\Andre\\data\\Ashleigh fallopian tube\\fallopian tubes\\AJER376\\fallopian_tubes_AJER376_013.ndpi', '\\\\10.162.80.16\\Andre\\data\\Ashleigh fallopian tube\\fallopian tubes\\AJER376\\fallopian_tubes_AJER376_015.ndpi', '\\\\10.162.80.16\\Andre\\data\\Ashleigh fallopian tube\\fallopian tubes\\AJER376\\fallopian_tubes_AJER376_017.ndpi', '\\\\10.162.80.16\\Andre\\data\\Ashleigh fallopian tube\\fallopian tubes\\AJER376\\fallopian_tubes_AJER376_019.ndpi', '\\\\10.162.80.16\\Andre\\data\\Ashleigh fallopian tube\\fallopian tubes\\AJER376\\fallopian_tubes_AJER376_021.ndpi', '\\\\10.162.80.16\\Andre\\data\\Ashleigh fallopian tube

In [ ]:
# Segment all WSIs -- takes about 2-5 minutes per whole slide image to segment, about 3 minutes to save geojson file
for k, file_list in enumerate(WSIs):
    pth_folder = pths[k]
    # print(pth_folder)
    
    for p, img_pth in enumerate(file_list):
        
        out_pth = os.path.join(pth_folder,f'StarDist_{date}')
        out_pth_json = os.path.join(out_pth,'json')
        out_pth_tif = os.path.join(out_pth,'tif')

        try:
            name = os.path.basename(img_pth)
            print(f'{name} ({p+1}/{len(file_list)})')
            if not os.path.exists(os.path.join(out_pth_json, (name[:-5] + '.json'))):
                # print(f'Starting {name}')
    
                img = imread(img_pth)
                img = img/255  # normalization used to train model
                _, polys = model.predict_instances_big(img, axes='YXC', block_size=4096, min_overlap=128, context=128, n_tiles=(4,4,1))
    
                print('Saving json...')
                save_json(polys, out_pth_json, name)
    
                # tif file is like 3 GB usually, so only uncomment next part if you are ok with that
                #print('Saving tif...')
                #imwrite(os.path.join(out_pth_tif, name[:-5] + '.tif'), labels)
            else:
                print(f'Skipping {name}')
        except:
            print(f'skipping {img_pth}, probably bc its too big...')

fallopian_tubes_AJER376_001.ndpi (1/288)
Skipping fallopian_tubes_AJER376_001.ndpi
fallopian_tubes_AJER376_003.ndpi (2/288)
Skipping fallopian_tubes_AJER376_003.ndpi
fallopian_tubes_AJER376_009.ndpi (3/288)
Skipping fallopian_tubes_AJER376_009.ndpi
fallopian_tubes_AJER376_013.ndpi (4/288)
Skipping fallopian_tubes_AJER376_013.ndpi
fallopian_tubes_AJER376_015.ndpi (5/288)
Skipping fallopian_tubes_AJER376_015.ndpi
fallopian_tubes_AJER376_017.ndpi (6/288)
Skipping fallopian_tubes_AJER376_017.ndpi
fallopian_tubes_AJER376_019.ndpi (7/288)
Skipping fallopian_tubes_AJER376_019.ndpi
fallopian_tubes_AJER376_021.ndpi (8/288)
Skipping fallopian_tubes_AJER376_021.ndpi
fallopian_tubes_AJER376_023.ndpi (9/288)
Skipping fallopian_tubes_AJER376_023.ndpi
fallopian_tubes_AJER376_025.ndpi (10/288)
Skipping fallopian_tubes_AJER376_025.ndpi
fallopian_tubes_AJER376_027.ndpi (11/288)
Skipping fallopian_tubes_AJER376_027.ndpi
fallopian_tubes_AJER376_029.ndpi (12/288)
Skipping fallopian_tubes_AJER376_029.ndpi
f

100%|██████████| 336/336 [13:16<00:00,  2.37s/it]


Saving json...
Finished fallopian_tubes_AJGB283_085.json
fallopian_tubes_AJGB283_087.ndpi (30/300)
effective: block_size=(4096, 4096, 3), min_overlap=(128, 128, 0), context=(128, 128, 0)


100%|██████████| 336/336 [12:56<00:00,  2.31s/it]


Saving json...
Finished fallopian_tubes_AJGB283_087.json
fallopian_tubes_AJGB283_089.ndpi (31/300)
effective: block_size=(4096, 4096, 3), min_overlap=(128, 128, 0), context=(128, 128, 0)


...

100%|██████████| 352/352 [1:01:19<00:00, 10.45s/it]


Saving json...
Finished fallopian_tubes_AJGB283_147.json
fallopian_tubes_AJGB283_149.ndpi (61/300)
effective: block_size=(4096, 4096, 3), min_overlap=(128, 128, 0), context=(128, 128, 0)


100%|██████████| 352/352 [1:02:44<00:00, 10.69s/it]


Saving json...
Finished fallopian_tubes_AJGB283_149.json
fallopian_tubes_AJGB283_151.ndpi (62/300)
effective: block_size=(4096, 4096, 3), min_overlap=(128, 128, 0), context=(128, 128, 0)


 88%|████████▊ | 309/352 [57:18<08:25, 11.76s/it] 

In [ ]:
# show_image(labels, crop_x=10000, crop_y=10000, tile_size=1024, cmap=cmap)
# show_image(img, crop_x, crop_y, tile_size, cmap=cmap)